In [68]:
import pandas as pd
import numpy as np
import os

pd.set_option('display.max_colwidth', None)

In [69]:
shake_bi = pd.read_csv('../data/shakespearean_bigrams.csv')
shake_tri = pd.read_csv('../data/shakespearean_trigrams.csv')
shake_four = pd.read_csv('../data/shakespearean_fourgrams.csv')

In [70]:
shake_bi.sample(5)

,title,title_tokenized,title_pos,pos_tag
44492,"corruption ,","['corruption', ',']","[('corruption', 'NN'), (',', ',')]","['NN', ',']"
233581,which very,"['which', 'very']","[('which', 'WDT'), ('very', 'RB')]","['WDT', 'RB']"
103903,into lust,"['into', 'lust']","[('into', 'IN'), ('lust', 'NN')]","['IN', 'NN']"
83906,hast left,"['hast', 'left']","[('hast', 'NN'), ('left', 'VBD')]","['NN', 'VBD']"
176342,shrink up,"['shrink', 'up']","[('shrink', 'VB'), ('up', 'RP')]","['VB', 'RP']"


In [71]:
token_count = []

for title in shake_bi['title_tokenized']:
    token_count.append(len(title))
    
shake_bi['tokenized_len'] = token_count
shake_bi.sample(5)

,title,title_tokenized,title_pos,pos_tag,tokenized_len
165729,revenges burn,"['revenges', 'burn']","[('revenges', 'NNS'), ('burn', 'VBP')]","['NNS', 'VBP']",20
61855,even fall,"['even', 'fall']","[('even', 'RB'), ('fall', 'VB')]","['RB', 'VB']",16
238960,with thee,"['with', 'thee']","[('with', 'IN'), ('thee', 'NN')]","['IN', 'NN']",16
127556,more rage,"['more', 'rage']","[('more', 'RBR'), ('rage', 'NN')]","['RBR', 'NN']",16
48443,day smile,"['day', 'smile']","[('day', 'NN'), ('smile', 'VB')]","['NN', 'VB']",16


I first tried to count the number of POS tags in `pos_tag` and got similar results- the entire list is stored as a string. The code below will convert the `pos tag` column from a string of a list of strings to simply a list of strings.

In [72]:
bi_list = shake_bi.pos_tag.tolist()

In [73]:
bi_list[5000:5005]

["['NN', 'MD']",
 "['NN', 'VBD']",
 "['NN', 'NNS']",
 "['NN', 'IN']",
 "['NN', 'NN']"]

In [74]:
new_bi_list = [(b.replace("[","").replace("]", "").replace("'","").split(", ")) for b in bi_list]

In [75]:
new_bi_list[5000:5005]

[['NN', 'MD'], ['NN', 'VBD'], ['NN', 'NNS'], ['NN', 'IN'], ['NN', 'NN']]

In [76]:
shake_bi['pos'] = new_bi_list
shake_bi.sample(5)

,title,title_tokenized,title_pos,pos_tag,tokenized_len,pos
59083,elder days,"['elder', 'days']","[('elder', 'NN'), ('days', 'NNS')]","['NN', 'NNS']",17,"[NN, NNS]"
118371,lusty widow,"['lusty', 'widow']","[('lusty', 'JJ'), ('widow', 'NN')]","['JJ', 'NN']",18,"[JJ, NN]"
50248,"den ,","['den', ',']","[('den', 'NN'), (',', ',')]","['NN', ',']",12,"[NN, ,]"
135345,night ?,"['night', '?']","[('night', 'NN'), ('?', '.')]","['NN', '.']",14,"[NN, .]"
246082,your kindest,"['your', 'kindest']","[('your', 'PRP$'), ('kindest', 'JJS')]","['PRP$', 'JJS']",19,"[PRP$, JJS]"


In [77]:
pos_count = []

for pos in shake_bi['pos']:
    pos_count.append(len(pos))
    
shake_bi['pos_len'] = pos_count
shake_bi.sample(5)

,title,title_tokenized,title_pos,pos_tag,tokenized_len,pos,pos_len
232199,when he,"['when', 'he']","[('when', 'WRB'), ('he', 'PRP')]","['WRB', 'PRP']",14,"[WRB, PRP]",2
220573,two short,"['two', 'short']","[('two', 'CD'), ('short', 'JJ')]","['CD', 'JJ']",16,"[CD, JJ]",2
86619,he express,"['he', 'express']","[('he', 'PRP'), ('express', 'VB')]","['PRP', 'VB']",17,"[PRP, VB]",2
127155,more continuate,"['more', 'continuate']","[('more', 'RBR'), ('continuate', 'NN')]","['RBR', 'NN']",22,"[RBR, NN]",2
151795,paris went,"['paris', 'went']","[('paris', 'NN'), ('went', 'VBD')]","['NN', 'VBD']",17,"[NN, VBD]",2


In [79]:
shake_bi.drop(columns=['title_pos', 'tokenized_len'], inplace=True)
shake_bi.head()

,title,title_tokenized,pos_tag,pos,pos_len
0,1. a,"['1.', 'a']","['CD', 'DT']","[CD, DT]",2
1,10. certain,"['10.', 'certain']","['CD', 'JJ']","[CD, JJ]",2
2,2. then,"['2.', 'then']","['CD', 'RB']","[CD, RB]",2
3,2d .,"['2d', '.']","['CD', '.']","[CD, .]",2
4,2s. 2d,"['2s.', '2d']","['CD', 'CD']","[CD, CD]",2


In [80]:
shake_bi.pos_len.value_counts()

2    242601
3      4707
4        57
Name: pos_len, dtype: int64

I suspected that punctuation would cause 2+ POS tags. I will remove these, and consider adding to the tri- and fourgram datasets

In [81]:
tri_not_bi = shake_bi[shake_bi['pos_len']==3]
tri_not_bi.shape

(4707, 5)

In [82]:
four_not_bi = shake_bi[shake_bi['pos_len']==4]
four_not_bi.shape

(57, 5)

In [84]:
tri_not_bi.sample(5)

,title,title_tokenized,pos_tag,pos,pos_len
157013,potent art.,"['potent', 'art', '.']","['JJ', 'NN', '.']","[JJ, NN, .]",3
171217,see one.,"['see', 'one', '.']","['VB', 'CD', '.']","[VB, CD, .]",3
197026,the fire.,"['the', 'fire', '.']","['DT', 'NN', '.']","[DT, NN, .]",3
119321,make faults.,"['make', 'faults', '.']","['NN', 'NNS', '.']","[NN, NNS, .]",3
195224,the 'scape.,"['the', ""'scape"", '.']","['DT', 'NN', '.']","[DT, NN, .]",3


In [86]:
four_not_bi.sample(5)

,title,title_tokenized,pos_tag,pos,pos_len
247286,youth suffer'd.,"['youth', 'suffer', ""'d"", '.']","['NN', 'NN', 'POS', '.']","[NN, NN, POS, .]",4
80406,great p's.,"['great', 'p', ""'s"", '.']","['JJ', 'NN', 'POS', '.']","[JJ, NN, POS, .]",4
212420,thy master's.,"['thy', 'master', ""'s"", '.']","['JJ', 'NN', 'POS', '.']","[JJ, NN, POS, .]",4
51484,diamonds dropp'd.,"['diamonds', 'dropp', ""'d"", '.']","['NNS', 'JJ', 'MD', '.']","[NNS, JJ, MD, .]",4
19175,as paul's.,"['as', 'paul', ""'s"", '.']","['IN', 'NN', 'POS', '.']","[IN, NN, POS, .]",4


In [102]:
pos_list = shake_bi.pos.tolist()
pos_list[6000:6005]

[['JJ', 'VBZ'], ['JJ', 'NN'], ['JJ', ','], ['IN', '.'], ['IN', '""']]

In [107]:
pos1 = []
pos2 = []

for i in pos_list:
    pos1.append(i[0])
    pos2.append(i[1])
    
shake_bi['pos1'] = pos1
shake_bi['pos2'] = pos2

shake_bi.sample(5)

,title,title_tokenized,pos_tag,pos,pos_len,pos1,pos2
232077,"when ,","['when', ',']","['WRB', ',']","[WRB, ,]",2,WRB,","
172022,senators descend,"['senators', 'descend']","['NNS', 'VBP']","[NNS, VBP]",2,NNS,VBP
133212,named sebastian,"['named', 'sebastian']","['VBN', 'JJ']","[VBN, JJ]",2,VBN,JJ
39105,cheerly on,"['cheerly', 'on']","['RB', 'IN']","[RB, IN]",2,RB,IN
59882,ends this,"['ends', 'this']","['NNS', 'DT']","[NNS, DT]",2,NNS,DT


Do the same thing with the the trigrams

In [109]:
shake_tri.sample(5)

,title,title_tokenized,title_pos,pos_tag
187034,i give thy,"['i', 'give', 'thy']","[('i', 'RB'), ('give', 'VBP'), ('thy', 'NN')]","['RB', 'VBP', 'NN']"
43862,as it appears,"['as', 'it', 'appears']","[('as', 'IN'), ('it', 'PRP'), ('appears', 'VBZ')]","['IN', 'PRP', 'VBZ']"
183879,i ' faith,"['i', ""'"", 'faith']","[('i', 'NN'), (""'"", ""''""), ('faith', 'NN')]","['NN', ""''"", 'NN']"
296568,"or dane ,","['or', 'dane', ',']","[('or', 'CC'), ('dane', 'NN'), (',', ',')]","['CC', 'NN', ',']"
280829,o strange !,"['o', 'strange', '!']","[('o', 'JJ'), ('strange', 'NN'), ('!', '.')]","['JJ', 'NN', '.']"


In [110]:
tri_list = shake_tri.pos_tag.tolist()

In [112]:
tri_list[7000:7005]

["['DT', 'NN', 'POS']",
 "['DT', 'NN', ',']",
 "['DT', 'NN', '.']",
 "['DT', 'JJ', '.']",
 "['DT', 'JJ', 'NN']"]

In [113]:
new_tri_list = [(t.replace("[","").replace("]", "").replace("'","").split(", ")) for t in tri_list]

In [114]:
new_tri_list[7000:7005]

[['DT', 'NN', 'POS'],
 ['DT', 'NN', ','],
 ['DT', 'NN', '.'],
 ['DT', 'JJ', '.'],
 ['DT', 'JJ', 'NN']]

In [115]:
shake_tri['pos'] = new_tri_list
shake_tri.sample(5)

,title,title_tokenized,title_pos,pos_tag,pos
228673,"little axe ,","['little', 'axe', ',']","[('little', 'JJ'), ('axe', 'NN'), (',', ',')]","['JJ', 'NN', ',']","[JJ, NN, ,]"
268708,no countermand for,"['no', 'countermand', 'for']","[('no', 'DT'), ('countermand', 'NN'), ('for', 'IN')]","['DT', 'NN', 'IN']","[DT, NN, IN]"
473117,with those infirmities,"['with', 'those', 'infirmities']","[('with', 'IN'), ('those', 'DT'), ('infirmities', 'NNS')]","['IN', 'DT', 'NNS']","[IN, DT, NNS]"
3411,a double majesty,"['a', 'double', 'majesty']","[('a', 'DT'), ('double', 'JJ'), ('majesty', 'NN')]","['DT', 'JJ', 'NN']","[DT, JJ, NN]"
440989,v. london. the,"['v.', 'london.', 'the']","[('v.', 'NN'), ('london.', 'VBZ'), ('the', 'DT')]","['NN', 'VBZ', 'DT']","[NN, VBZ, DT]"


In [116]:
pos_count = []

for pos in shake_tri['pos']:
    pos_count.append(len(pos))
    
shake_tri['pos_len'] = pos_count
shake_tri.sample(5)

,title,title_tokenized,title_pos,pos_tag,pos,pos_len
48228,aufidius. marcius fights,"['aufidius.', 'marcius', 'fights']","[('aufidius.', 'NN'), ('marcius', 'NN'), ('fights', 'NNS')]","['NN', 'NN', 'NNS']","[NN, NN, NNS]",3
19190,"am proud ,","['am', 'proud', ',']","[('am', 'VBP'), ('proud', 'NN'), (',', ',')]","['VBP', 'NN', ',']","[VBP, NN, ,]",3
227224,like angels of,"['like', 'angels', 'of']","[('like', 'IN'), ('angels', 'NNS'), ('of', 'IN')]","['IN', 'NNS', 'IN']","[IN, NNS, IN]",3
89765,country from calamity,"['country', 'from', 'calamity']","[('country', 'NN'), ('from', 'IN'), ('calamity', 'NN')]","['NN', 'IN', 'NN']","[NN, IN, NN]",3
491109,your oaths are,"['your', 'oaths', 'are']","[('your', 'PRP$'), ('oaths', 'NNS'), ('are', 'VBP')]","['PRP$', 'NNS', 'VBP']","[PRP$, NNS, VBP]",3


In [117]:
shake_tri.pos_len.value_counts()

3    488003
4      4995
5        55
Name: pos_len, dtype: int64

In [118]:
shake_tri.drop(columns=['title_pos'], inplace=True)

In [120]:
four_not_tri = shake_tri[shake_tri['pos_len']==4]
four_not_tri.shape

(4995, 5)

In [121]:
five_not_tri = shake_tri[shake_tri['pos_len']==5]
five_not_tri.shape

(55, 5)

In [122]:
shake_tri.pos_len.value_counts()

3    488003
4      4995
5        55
Name: pos_len, dtype: int64

In [123]:
shake_tri = shake_tri[shake_tri['pos_len']==3]
shake_tri.shape

(488003, 5)

In [124]:
shake_bi.pos_len.value_counts()

2    242601
3      4707
4        57
Name: pos_len, dtype: int64

In [125]:
shake_bi = shake_bi[shake_bi['pos_len']==2]
shake_bi.shape

(242601, 7)

In [128]:
shake_tri = shake_tri.append(tri_not_bi).reset_index(drop=True)
shake_tri.shape

(492710, 5)

In [129]:
shake_tri.pos_len.value_counts()

3    492710
Name: pos_len, dtype: int64

In [145]:
pos_list = shake_tri.pos.tolist()
pos_list[6000:6005]

[['DT', 'NN', ','],
 ['DT', 'NN', ':'],
 ['DT', 'NN', '.'],
 ['DT', 'NN', '.'],
 ['DT', 'NN', 'DT']]

In [146]:
pos1 = []
pos2 = []
pos3 = []

for i in pos_list:
    pos1.append(i[0])
    pos2.append(i[1])
    pos3.append(i[2])
    
shake_tri['pos1'] = pos1
shake_tri['pos2'] = pos2
shake_tri['pos3'] = pos3

shake_tri.sample(5)

,title,title_tokenized,pos_tag,pos,pos_len,pos1,pos2,pos3
288860,often made against,"['often', 'made', 'against']","['RB', 'VBN', 'IN']","[RB, VBN, IN]",3,RB,VBN,IN
124105,"fit , else","['fit', ',', 'else']","['NN', ',', 'RB']","[NN, ,, RB]",3,NN,",",RB
2359,a canopy most,"['a', 'canopy', 'most']","['DT', 'NN', 'RBS']","[DT, NN, RBS]",3,DT,NN,RBS
43841,as my thoughts,"['as', 'my', 'thoughts']","['IN', 'PRP$', 'NNS']","[IN, PRP$, NNS]",3,IN,PRP$,NNS
98658,did this companion,"['did', 'this', 'companion']","['VBD', 'DT', 'NN']","[VBD, DT, NN]",3,VBD,DT,NN


And now the fourgrams

In [133]:
shake_four.sample(5)

,title,title_tokenized,title_pos,pos_tag
30470,"and losing her ,","['and', 'losing', 'her', ',']","[('and', 'CC'), ('losing', 'VBG'), ('her', 'PRP'), (',', ',')]","['CC', 'VBG', 'PRP', ',']"
270674,"more , be sage","['more', ',', 'be', 'sage']","[('more', 'RBR'), (',', ','), ('be', 'VB'), ('sage', 'NN')]","['RBR', ',', 'VB', 'NN']"
289821,"noble charles , the","['noble', 'charles', ',', 'the']","[('noble', 'JJ'), ('charles', 'NNS'), (',', ','), ('the', 'DT')]","['JJ', 'NNS', ',', 'DT']"
78545,by break of day,"['by', 'break', 'of', 'day']","[('by', 'IN'), ('break', 'NN'), ('of', 'IN'), ('day', 'NN')]","['IN', 'NN', 'IN', 'NN']"
294029,not here. that you,"['not', 'here.', 'that', 'you']","[('not', 'RB'), ('here.', 'PDT'), ('that', 'IN'), ('you', 'PRP')]","['RB', 'PDT', 'IN', 'PRP']"


In [134]:
four_list = shake_four.pos_tag.tolist()

four_list[78545:78550]

["['IN', 'NN', 'IN', 'NN']",
 "['IN', 'VBG', 'IN', 'DT']",
 "['IN', 'NN', 'VBZ', 'JJ']",
 "['IN', 'NN', 'IN', 'PRP$']",
 "['IN', 'JJ', 'NNS', ',']"]

In [135]:
new_four_list = [(f.replace("[","").replace("]", "").replace("'","").split(", ")) for f in four_list]

new_four_list[78545:78550]

[['IN', 'NN', 'IN', 'NN'],
 ['IN', 'VBG', 'IN', 'DT'],
 ['IN', 'NN', 'VBZ', 'JJ'],
 ['IN', 'NN', 'IN', 'PRP$'],
 ['IN', 'JJ', 'NNS', ',']]

In [136]:
shake_four['pos'] = new_four_list
shake_four.sample(5)

,title,title_tokenized,title_pos,pos_tag,pos
299692,now newly performed by,"['now', 'newly', 'performed', 'by']","[('now', 'RB'), ('newly', 'RB'), ('performed', 'VBN'), ('by', 'IN')]","['RB', 'RB', 'VBN', 'IN']","[RB, RB, VBN, IN]"
18530,"aloft. enter , below","['aloft.', 'enter', ',', 'below']","[('aloft.', 'NN'), ('enter', 'NN'), (',', ','), ('below', 'IN')]","['NN', 'NN', ',', 'IN']","[NN, NN, ,, IN]"
156145,"hang over thee ,","['hang', 'over', 'thee', ',']","[('hang', 'NN'), ('over', 'IN'), ('thee', 'NN'), (',', ',')]","['NN', 'IN', 'NN', ',']","[NN, IN, NN, ,]"
24389,and by good fortune,"['and', 'by', 'good', 'fortune']","[('and', 'CC'), ('by', 'IN'), ('good', 'JJ'), ('fortune', 'NN')]","['CC', 'IN', 'JJ', 'NN']","[CC, IN, JJ, NN]"
107191,do love it better,"['do', 'love', 'it', 'better']","[('do', 'VB'), ('love', 'VB'), ('it', 'PRP'), ('better', 'JJR')]","['VB', 'VB', 'PRP', 'JJR']","[VB, VB, PRP, JJR]"


In [137]:
pos_count = []

for pos in shake_four['pos']:
    pos_count.append(len(pos))
    
shake_four['pos_len'] = pos_count
shake_four.sample(5)

,title,title_tokenized,title_pos,pos_tag,pos,pos_len
265067,mean i to enrich,"['mean', 'i', 'to', 'enrich']","[('mean', 'NN'), ('i', 'NN'), ('to', 'TO'), ('enrich', 'VB')]","['NN', 'NN', 'TO', 'VB']","[NN, NN, TO, VB]",4
78474,by and scarcely bruise,"['by', 'and', 'scarcely', 'bruise']","[('by', 'IN'), ('and', 'CC'), ('scarcely', 'RB'), ('bruise', 'VB')]","['IN', 'CC', 'RB', 'VB']","[IN, CC, RB, VB]",4
336060,"proportion , didst bring","['proportion', ',', 'didst', 'bring']","[('proportion', 'NN'), (',', ','), ('didst', 'NN'), ('bring', 'NN')]","['NN', ',', 'NN', 'NN']","[NN, ,, NN, NN]",4
251320,love you ? ',"['love', 'you', '?', ""'""]","[('love', 'VB'), ('you', 'PRP'), ('?', '.'), (""'"", ""''"")]","['VB', 'PRP', '.', ""''""]","[VB, PRP, ., """"]",4
302392,o'er some high-viced city,"[""o'er"", 'some', 'high-viced', 'city']","[(""o'er"", 'VB'), ('some', 'DT'), ('high-viced', 'JJ'), ('city', 'NN')]","['VB', 'DT', 'JJ', 'NN']","[VB, DT, JJ, NN]",4


In [138]:
shake_four.drop(columns=['title_pos'], inplace=True)

shake_four.pos_len.value_counts()

4    523350
5      4310
6        45
Name: pos_len, dtype: int64

In [139]:
shake_four = shake_four[shake_four['pos_len']==4]
shake_four.shape

(523350, 5)

In [140]:
shake_four = shake_four.append(four_not_bi).reset_index(drop=True)
shake_four.shape

(523407, 5)

In [141]:
shake_four = shake_four.append(four_not_tri).reset_index(drop=True)
shake_four.shape

(528402, 5)

In [142]:
shake_four.pos_len.value_counts()

4    528402
Name: pos_len, dtype: int64

In [152]:
pos_list = shake_four.pos.tolist()
pos_list[6000:6005]

[['DT', 'NN', ',', 'NN'],
 ['DT', 'NN', ',', 'RB'],
 ['DT', 'NN', ',', 'RB'],
 ['DT', 'NN', ',', 'NN'],
 ['DT', 'NN', ',', 'IN']]

In [153]:
pos1 = []
pos2 = []
pos3 = []
pos4 = []

for i in pos_list:
    pos1.append(i[0])
    pos2.append(i[1])
    pos3.append(i[2])
    pos4.append(i[3])
    
shake_four['pos1'] = pos1
shake_four['pos2'] = pos2
shake_four['pos3'] = pos3
shake_four['pos4'] = pos4

shake_four.sample(5)

,title,title_tokenized,pos_tag,pos,pos_len,pos1,pos2,pos3,pos4
387150,"that , and tell","['that', ',', 'and', 'tell']","['DT', ',', 'CC', 'VB']","[DT, ,, CC, VB]",4,DT,",",CC,VB
499091,with message unto princely,"['with', 'message', 'unto', 'princely']","['IN', 'NN', 'NNS', 'RB']","[IN, NN, NNS, RB]",4,IN,NN,NNS,RB
323128,part in this fair,"['part', 'in', 'this', 'fair']","['NN', 'IN', 'DT', 'NN']","[NN, IN, DT, NN]",4,NN,IN,DT,NN
106880,do poor tom some,"['do', 'poor', 'tom', 'some']","['VB', 'JJ', 'VB', 'DT']","[VB, JJ, VB, DT]",4,VB,JJ,VB,DT
354823,shalt find a king,"['shalt', 'find', 'a', 'king']","['NN', 'VB', 'DT', 'NN']","[NN, VB, DT, NN]",4,NN,VB,DT,NN


In [154]:
shake_tri.sample(5)

,title,title_tokenized,pos_tag,pos,pos_len,pos1,pos2,pos3
393744,there upon inter'gatories,"['there', 'upon', ""inter'gatories""]","['RB', 'IN', 'NNS']","[RB, IN, NNS]",3,RB,IN,NNS
136865,gain of my,"['gain', 'of', 'my']","['NN', 'IN', 'PRP$']","[NN, IN, PRP$]",3,NN,IN,PRP$
351928,stands chief in,"['stands', 'chief', 'in']","['NNS', 'NN', 'IN']","[NNS, NN, IN]",3,NNS,NN,IN
245651,merit bliss by,"['merit', 'bliss', 'by']","['NN', 'NN', 'IN']","[NN, NN, IN]",3,NN,NN,IN
18136,am a fellow,"['am', 'a', 'fellow']","['VBP', 'DT', 'NN']","[VBP, DT, NN]",3,VBP,DT,NN


In [156]:
shake_bi.sample(5)

,title,title_tokenized,pos_tag,pos,pos_len,pos1,pos2
107113,it. go,"['it.', 'go']","['NN', 'VB']","[NN, VB]",2,NN,VB
22207,base second,"['base', 'second']","['NN', 'NN']","[NN, NN]",2,NN,NN
68236,finds itself,"['finds', 'itself']","['NNS', 'PRP']","[NNS, PRP]",2,NNS,PRP
124558,merry. yea,"['merry.', 'yea']","['NN', 'NN']","[NN, NN]",2,NN,NN
193314,that came,"['that', 'came']","['DT', 'VBD']","[DT, VBD]",2,DT,VBD


In [157]:
datapath = '../data'

datapath_shake_bi = os.path.join(datapath, 'shakespearean_bigrams_pos.csv')
shake_bi.to_csv(datapath_shake_bi, index=False)

In [158]:
datapath_shake_tri = os.path.join(datapath, 'shakespearean_trigrams_pos.csv')
shake_tri.to_csv(datapath_shake_tri, index=False)

In [159]:
datapath_shake_four = os.path.join(datapath, 'shakespearean_fourgrams_pos.csv')
shake_four.to_csv(datapath_shake_four, index=False)